# US Patent Phrase to Phrase Matching – Plan

Goal: Achieve medal-level Pearson on public LB by building a strong, validated text similarity model with robust CV and iterative improvements.

Workflow:
- Environment check (GPU sanity) and data load
- EDA: target distribution, text lengths, CPC distributions
- Validation: GroupKFold by anchor or stratified bins on score; fix a single protocol
- Baseline: TF-IDF + Ridge/SGDRegressor; evaluate Pearson OOF
- Strong models:
  - Sentence-transformers (e.g., msmarco-distilbert-base-v4 / bge-small-en) finetune with cosine-sim regression
  - Cross-encoder (e.g., MiniLM-L12-v2) regression head
- Features:
  - Text normalization, lowercasing, dedup punctuation
  - Use CPC code as domain context (prefix to text)
  - Pairwise features: BM25/TFIDF cosine; embedding cosine; length ratios
- Ensembling:
  - Blend TF-IDF ridge, bi-encoder cosine, cross-encoder regression
- Error analysis: mine worst OOF buckets by CPC and length; iterate
- Submission: Generate test predictions; save submission.csv

Major checkpoints for expert review:
1) After this plan and environment check
2) After CV protocol + baseline OOF
3) After first transformer model OOF
4) Before long training or ensembling

Logging: Print elapsed time and fold indices; cache OOF/test predictions to reuse.

In [ ]:
# Environment check and quick data scan
import os, sys, time, shutil, subprocess, json, textwrap
import pandas as pd, numpy as np
from pathlib import Path

t0 = time.time()
print('CWD:', os.getcwd())
print('Python:', sys.version)
print('Start time:', time.strftime('%Y-%m-%d %H:%M:%S'))

print('\n== nvidia-smi ==', flush=True)
subprocess.run(['bash','-lc','nvidia-smi || true'], check=False)

data_dir = Path('.')
train_path = data_dir/'train.csv'
test_path = data_dir/'test.csv'
assert train_path.exists() and test_path.exists(), 'Missing train/test CSVs'

usecols_train = None
train = pd.read_csv(train_path, usecols=usecols_train)
test = pd.read_csv(test_path)
print(f'Loaded train: {train.shape}, test: {test.shape}')
print('Train columns:', train.columns.tolist())
print('Test columns:', test.columns.tolist())

# Basic target and text stats
if 'score' in train.columns:
    s = train['score'].astype(float)
    print('Score describe:\n', s.describe())
    hist_counts, hist_bins = np.histogram(s, bins=10, range=(0,1))
    print('Score hist counts:', hist_counts.tolist())
    print('Score hist bins:', np.round(hist_bins,3).tolist())

def text_len_stats(df, cols):
    out = {}
    for c in cols:
        if c in df.columns:
            l = df[c].astype(str).str.len()
            out[c] = dict(count=int(l.count()), mean=float(l.mean()), p50=float(l.median()), p95=float(l.quantile(0.95)), max=int(l.max()))
    return out

txt_cols = ['anchor','target','context']
print('Train text len stats:', json.dumps(text_len_stats(train, txt_cols), indent=2))
print('Test  text len stats:', json.dumps(text_len_stats(test, txt_cols), indent=2))

if 'context' in train.columns:
    print('Top contexts (train):')
    print(train['context'].value_counts().head(10))
if 'anchor' in train.columns:
    print('Unique anchors:', train['anchor'].nunique())

print('Elapsed: %.2fs' % (time.time()-t0))

In [ ]:
# Build deduped data and 5-fold GroupKFold by anchor; cache folds
import unicodedata
from sklearn.model_selection import GroupKFold

t0 = time.time()

def normalize_text(s: str) -> str:
    if pd.isna(s):
        return ''
    s = unicodedata.normalize('NFKC', str(s))
    s = s.lower()
    # collapse multiple spaces
    s = ' '.join(s.split())
    # simple punctuation de-dup (keep hyphens/slashes)
    for ch in [',', '.', ';', ':', '!', '?', '(', ')', '[', ']', '{', '}', "'", '"']:
        while ch*2 in s:
            s = s.replace(ch*2, ch)
    return s

# Prepare normalized columns (do not overwrite originals; folds use original anchor as group)
train['_anchor_n'] = train['anchor'].map(normalize_text)
train['_target_n'] = train['target'].map(normalize_text)
train['_context_n'] = train['context'].map(normalize_text)

# Exact-duplicate handling on original triplets (anchor, target, context)
key_cols = ['anchor','target','context']
train['_key'] = (train['anchor'].astype(str) + '\t' + train['target'].astype(str) + '\t' + train['context'].astype(str))

# Map from original row to dedup index
keys, inv = np.unique(train['_key'].values, return_inverse=True)
train['_dedup_idx'] = inv

# Aggregate scores per dedup_idx
agg = train.groupby('_dedup_idx', as_index=False).agg({
    'id':'first',
    'anchor':'first',
    'target':'first',
    'context':'first',
    '_anchor_n':'first',
    '_target_n':'first',
    '_context_n':'first',
    'score':'mean',
})
agg = agg.rename(columns={'score':'score_mean'})
agg = agg.sort_values('_dedup_idx').reset_index(drop=True)

# Create 5-fold GroupKFold by anchor (original anchor string)
n_splits = 5
gkf = GroupKFold(n_splits=n_splits)
groups = agg['anchor'].astype(str).values
folds = np.full(len(agg), -1, dtype=int)
for f, (tr_idx, va_idx) in enumerate(gkf.split(np.arange(len(agg)), groups=groups)):
    folds[va_idx] = f
agg['fold'] = folds
assert (agg['fold']>=0).all(), 'Fold assignment failed'

# Persist artifacts
Path('artifacts').mkdir(exist_ok=True)
agg[['anchor','target','context','_anchor_n','_target_n','_context_n','score_mean','fold','_dedup_idx']].to_csv('artifacts/train_dedup_folds.csv', index=False)
pd.DataFrame({'orig_index': np.arange(len(train)), 'dedup_idx': train['_dedup_idx'].values}).to_csv('artifacts/orig_to_dedup_map.csv', index=False)

# Log fold sizes and CPC distribution sanity
sizes = agg['fold'].value_counts().sort_index().to_dict()
print('Fold sizes (dedup space):', sizes)
print('Unique dedup rows:', len(agg), 'from original rows:', len(train))
print('Elapsed: %.2fs' % (time.time()-t0))

In [ ]:
# TF-IDF + Ridge baseline with cosine + length features; 5-fold GroupKFold; cache OOF/test and submission
import time
import numpy as np
import pandas as pd
from pathlib import Path
from scipy import sparse
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import Ridge
from sklearn.preprocessing import normalize as l2_normalize
from sklearn.metrics import r2_score
from scipy.stats import pearsonr

t0 = time.time()
print('Loading fold artifacts...', flush=True)
dedup = pd.read_csv('artifacts/train_dedup_folds.csv')
orig_map = pd.read_csv('artifacts/orig_to_dedup_map.csv')

def pair_text(df):
    return ('[CPC] ' + df['_context_n'].astype(str) + ' [A] ' + df['_anchor_n'].astype(str) + ' [B] ' + df['_target_n'].astype(str)).values

def rowwise_cosine(Xa, Xt):
    Xa = l2_normalize(Xa, axis=1, copy=False)
    Xt = l2_normalize(Xt, axis=1, copy=False)
    sim = (Xa.multiply(Xt)).sum(axis=1)
    return np.asarray(sim).ravel()

def build_dense_feats(df):
    la = df['_anchor_n'].astype(str).str.len().values.astype(np.float32)
    lt = df['_target_n'].astype(str).str.len().values.astype(np.float32)
    ratio = (la / np.maximum(1.0, lt)).astype(np.float32)
    adiff = np.abs(la - lt).astype(np.float32)
    return np.vstack([la, lt, ratio, adiff]).T

# Prepare test normalized columns
test['_anchor_n'] = test['anchor'].map(normalize_text)
test['_target_n'] = test['target'].map(normalize_text)
test['_context_n'] = test['context'].map(normalize_text)

y = dedup['score_mean'].values.astype(np.float32)
folds = dedup['fold'].values.astype(int)
n_folds = int(dedup['fold'].nunique())

oof = np.zeros(len(dedup), dtype=np.float32)
test_preds_folds = []

for f in range(n_folds):
    t_fold = time.time()
    tr_idx = np.where(folds != f)[0]
    va_idx = np.where(folds == f)[0]
    print(f'Fold {f} | train {len(tr_idx)} | valid {len(va_idx)}', flush=True)

    df_tr = dedup.iloc[tr_idx].reset_index(drop=True)
    df_va = dedup.iloc[va_idx].reset_index(drop=True)

    # Vectorizers for anchor/target (word + char) to compute cosine features
    vocab_corpus = pd.concat([df_tr['_anchor_n'], df_tr['_target_n']], axis=0).astype(str).values
    wvec = TfidfVectorizer(ngram_range=(1,2), min_df=2, max_features=200_000, analyzer='word')
    cvec = TfidfVectorizer(ngram_range=(3,5), min_df=2, max_features=300_000, analyzer='char_wb')
    wvec.fit(vocab_corpus)
    cvec.fit(vocab_corpus)

    # Transform anchor/target
    Xa_tr_w = wvec.transform(df_tr['_anchor_n'].astype(str).values)
    Xt_tr_w = wvec.transform(df_tr['_target_n'].astype(str).values)
    Xa_va_w = wvec.transform(df_va['_anchor_n'].astype(str).values)
    Xt_va_w = wvec.transform(df_va['_target_n'].astype(str).values)

    Xa_tr_c = cvec.transform(df_tr['_anchor_n'].astype(str).values)
    Xt_tr_c = cvec.transform(df_tr['_target_n'].astype(str).values)
    Xa_va_c = cvec.transform(df_va['_anchor_n'].astype(str).values)
    Xt_va_c = cvec.transform(df_va['_target_n'].astype(str).values)

    cos_tr_w = rowwise_cosine(Xa_tr_w, Xt_tr_w)[:, None]
    cos_va_w = rowwise_cosine(Xa_va_w, Xt_va_w)[:, None]
    cos_tr_c = rowwise_cosine(Xa_tr_c, Xt_tr_c)[:, None]
    cos_va_c = rowwise_cosine(Xa_va_c, Xt_va_c)[:, None]

    dense_tr = build_dense_feats(df_tr)
    dense_va = build_dense_feats(df_va)

    dense_tr_all = np.hstack([dense_tr, cos_tr_w, cos_tr_c]).astype(np.float32)
    dense_va_all = np.hstack([dense_va, cos_va_w, cos_va_c]).astype(np.float32)

    Xdense_tr = sparse.csr_matrix(dense_tr_all)
    Xdense_va = sparse.csr_matrix(dense_va_all)

    # Pair TF-IDF (word + char) on formatted string
    pair_tr = pair_text(df_tr)
    pair_va = pair_text(df_va)
    pair_te = pair_text(test)

    p_wvec = TfidfVectorizer(ngram_range=(1,2), min_df=2, max_features=300_000, analyzer='word')
    p_cvec = TfidfVectorizer(ngram_range=(3,5), min_df=2, max_features=400_000, analyzer='char_wb')
    Xpw_tr = p_wvec.fit_transform(pair_tr)
    Xpw_va = p_wvec.transform(pair_va)
    Xpc_tr = p_cvec.fit_transform(pair_tr)
    Xpc_va = p_cvec.transform(pair_va)

    # Final train/valid matrices
    X_tr = sparse.hstack([Xpw_tr, Xpc_tr, Xdense_tr], format='csr')
    X_va = sparse.hstack([Xpw_va, Xpc_va, Xdense_va], format='csr')

    y_tr = y[tr_idx]
    y_va = y[va_idx]

    model = Ridge(alpha=1.5, random_state=42)
    t_fit = time.time()
    model.fit(X_tr, y_tr)
    print(f'  F{f} fit done in {time.time()-t_fit:.2f}s, nfeat={X_tr.shape[1]:,}', flush=True)

    pred_va = model.predict(X_va)
    oof[va_idx] = pred_va
    pr = pearsonr(y_va, pred_va)[0]
    print(f'  F{f} Pearson: {pr:.6f}', flush=True)

    # Test features via current fold vectorizers
    Xpw_te = p_wvec.transform(pair_te)
    Xpc_te = p_cvec.transform(pair_te)
    dense_te = build_dense_feats(test)
    # For test cosine features reuse wvec/cvec on test normalized anchor/target
    Xa_te_w = wvec.transform(test['_anchor_n'].astype(str).values)
    Xt_te_w = wvec.transform(test['_target_n'].astype(str).values)
    Xa_te_c = cvec.transform(test['_anchor_n'].astype(str).values)
    Xt_te_c = cvec.transform(test['_target_n'].astype(str).values)
    cos_te_w = rowwise_cosine(Xa_te_w, Xt_te_w)[:, None]
    cos_te_c = rowwise_cosine(Xa_te_c, Xt_te_c)[:, None]
    dense_te_all = np.hstack([dense_te, cos_te_w, cos_te_c]).astype(np.float32)
    Xdense_te = sparse.csr_matrix(dense_te_all)
    X_te = sparse.hstack([Xpw_te, Xpc_te, Xdense_te], format='csr')
    pred_te = model.predict(X_te)
    test_preds_folds.append(pred_te.astype(np.float32))

    print(f'Fold {f} done in {time.time()-t_fold:.2f}s', flush=True)

# Clip to [0,1]
oof_clip = np.clip(oof, 0.0, 1.0)
oof_pearson = float(pearsonr(y, oof_clip)[0])
print(f'OOF Pearson (clipped): {oof_pearson:.6f}')

test_pred = np.mean(np.vstack(test_preds_folds), axis=0)
test_pred = np.clip(test_pred, 0.0, 1.0).astype(np.float32)

# Cache OOF/test
Path('artifacts').mkdir(exist_ok=True)
np.save('artifacts/oof_tfidf_ridge.npy', oof_clip)
np.save('artifacts/test_tfidf_ridge.npy', test_pred)
pd.DataFrame({'dedup_idx': np.arange(len(dedup)), 'oof': oof_clip}).to_csv('artifacts/oof_tfidf_ridge.csv', index=False)

# Expand OOF to original rows for diagnostics (mean per dedup_idx mapping)
oof_full = oof_clip[orig_map['dedup_idx'].values]
pd.DataFrame({'id': train['id'], 'oof': oof_full, 'score': train['score']}).to_csv('artifacts/oof_full_rows.csv', index=False)

# Build submission
sub = pd.DataFrame({'id': test['id'], 'score': test_pred})
sub.to_csv('submission.csv', index=False)
print('Saved submission.csv with shape', sub.shape)
print('Total elapsed: %.2fs' % (time.time()-t0))

In [ ]:
# Dense relational baseline: TF-IDF cosines + length features -> StandardScaler -> Ridge
import time
import numpy as np
import pandas as pd
from pathlib import Path
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import normalize as l2_normalize, StandardScaler
from sklearn.linear_model import Ridge
from scipy.stats import pearsonr

t0 = time.time()
print('Loading fold artifacts...', flush=True)
dedup = pd.read_csv('artifacts/train_dedup_folds.csv')
orig_map = pd.read_csv('artifacts/orig_to_dedup_map.csv')

# Ensure test normalized columns exist
if '_anchor_n' not in test.columns:
    test['_anchor_n'] = test['anchor'].map(normalize_text)
    test['_target_n'] = test['target'].map(normalize_text)
    test['_context_n'] = test['context'].map(normalize_text)

def side_text_a(df):
    return (df['_context_n'].astype(str) + ' ' + df['_anchor_n'].astype(str)).values
def side_text_t(df):
    return (df['_context_n'].astype(str) + ' ' + df['_target_n'].astype(str)).values

def rowwise_cosine(Xa, Xt):
    Xa = l2_normalize(Xa, axis=1, copy=False)
    Xt = l2_normalize(Xt, axis=1, copy=False)
    sim = (Xa.multiply(Xt)).sum(axis=1)
    return np.asarray(sim).ravel()

def build_len_feats(df):
    la = df['_anchor_n'].astype(str).str.len().values.astype(np.float32)
    lt = df['_target_n'].astype(str).str.len().values.astype(np.float32)
    absdiff = np.abs(la - lt).astype(np.float32)
    ratio_sym = (np.minimum(la, lt) / np.maximum(1.0, np.maximum(la, lt))).astype(np.float32)
    return la, lt, absdiff, ratio_sym

y = dedup['score_mean'].values.astype(np.float32)
folds = dedup['fold'].values.astype(int)
n_folds = int(dedup['fold'].nunique())

oof = np.zeros(len(dedup), dtype=np.float32)
test_preds_folds = []

for f in range(n_folds):
    t_fold = time.time()
    tr_idx = np.where(folds != f)[0]
    va_idx = np.where(folds == f)[0]
    df_tr = dedup.iloc[tr_idx].reset_index(drop=True)
    df_va = dedup.iloc[va_idx].reset_index(drop=True)
    print(f'Fold {f} | train {len(tr_idx)} | valid {len(va_idx)}', flush=True)

    sA_tr = side_text_a(df_tr)
    sT_tr = side_text_t(df_tr)
    sA_va = side_text_a(df_va)
    sT_va = side_text_t(df_va)
    sA_te = side_text_a(test)
    sT_te = side_text_t(test)

    # Fit vectorizers on train-fold corpus (anchors+targets with CPC prepended)
    corpus = np.concatenate([sA_tr, sT_tr], axis=0)
    wvec = TfidfVectorizer(analyzer='word', ngram_range=(1,2), min_df=2, max_features=100_000)
    cvec = TfidfVectorizer(analyzer='char_wb', ngram_range=(3,5), min_df=2, max_features=200_000)
    wvec.fit(corpus)
    cvec.fit(corpus)

    # Transform and compute cosines
    Xa_tr_w = wvec.transform(sA_tr); Xt_tr_w = wvec.transform(sT_tr)
    Xa_va_w = wvec.transform(sA_va); Xt_va_w = wvec.transform(sT_va)
    Xa_te_w = wvec.transform(sA_te); Xt_te_w = wvec.transform(sT_te)
    cos_tr_w = rowwise_cosine(Xa_tr_w, Xt_tr_w)
    cos_va_w = rowwise_cosine(Xa_va_w, Xt_va_w)
    cos_te_w = rowwise_cosine(Xa_te_w, Xt_te_w)

    Xa_tr_c = cvec.transform(sA_tr); Xt_tr_c = cvec.transform(sT_tr)
    Xa_va_c = cvec.transform(sA_va); Xt_va_c = cvec.transform(sT_va)
    Xa_te_c = cvec.transform(sA_te); Xt_te_c = cvec.transform(sT_te)
    cos_tr_c = rowwise_cosine(Xa_tr_c, Xt_tr_c)
    cos_va_c = rowwise_cosine(Xa_va_c, Xt_va_c)
    cos_te_c = rowwise_cosine(Xa_te_c, Xt_te_c)

    # Length features + simple interactions
    la_tr, lt_tr, ad_tr, rs_tr = build_len_feats(df_tr)
    la_va, lt_va, ad_va, rs_va = build_len_feats(df_va)
    la_te, lt_te, ad_te, rs_te = build_len_feats(test)

    X_tr = np.vstack([
        cos_tr_w, cos_tr_c, la_tr, lt_tr, ad_tr, rs_tr,
        (la_tr * cos_tr_w), (lt_tr * cos_tr_c)
    ]).T.astype(np.float32)
    X_va = np.vstack([
        cos_va_w, cos_va_c, la_va, lt_va, ad_va, rs_va,
        (la_va * cos_va_w), (lt_va * cos_va_c)
    ]).T.astype(np.float32)
    X_te = np.vstack([
        cos_te_w, cos_te_c, la_te, lt_te, ad_te, rs_te,
        (la_te * cos_te_w), (lt_te * cos_te_c)
    ]).T.astype(np.float32)

    scaler = StandardScaler(with_mean=True, with_std=True)
    X_tr_s = scaler.fit_transform(X_tr)
    X_va_s = scaler.transform(X_va)
    X_te_s = scaler.transform(X_te)

    y_tr = y[tr_idx]
    y_va = y[va_idx]

    model = Ridge(alpha=1.5, random_state=42)
    t_fit = time.time()
    model.fit(X_tr_s, y_tr)
    pred_va = model.predict(X_va_s)
    pr = pearsonr(y_va, pred_va)[0]
    print(f'  F{f} Pearson: {pr:.6f} (fit {time.time()-t_fit:.2f}s)', flush=True)
    oof[va_idx] = pred_va

    pred_te = model.predict(X_te_s).astype(np.float32)
    test_preds_folds.append(pred_te)
    print(f'Fold {f} done in {time.time()-t_fold:.2f}s', flush=True)

oof_clip = np.clip(oof, 0.0, 1.0)
oof_pr = float(pearsonr(y, oof_clip)[0])
print(f'OOF Pearson (clipped): {oof_pr:.6f}', flush=True)

test_pred = np.mean(np.vstack(test_preds_folds), axis=0)
test_pred = np.clip(test_pred, 0.0, 1.0).astype(np.float32)

Path('artifacts').mkdir(exist_ok=True)
np.save('artifacts/oof_dense_ridge.npy', oof_clip)
np.save('artifacts/test_dense_ridge.npy', test_pred)
pd.DataFrame({'dedup_idx': np.arange(len(dedup)), 'oof': oof_clip}).to_csv('artifacts/oof_dense_ridge.csv', index=False)

# Expand OOF for diagnostics
oof_full = oof_clip[orig_map['dedup_idx'].values]
pd.DataFrame({'id': train['id'], 'oof': oof_full, 'score': train['score']}).to_csv('artifacts/oof_dense_full_rows.csv', index=False)

# Build submission
pd.DataFrame({'id': test['id'], 'score': test_pred}).to_csv('submission.csv', index=False)
print('Saved submission.csv; Total elapsed: %.2fs' % (time.time()-t0))

In [ ]:
# Diagnostic: check raw cosine correlations (no model) to locate issue
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import normalize as l2_normalize
from scipy.stats import pearsonr

def side_text_a(df):
    return (df['_context_n'].astype(str) + ' ' + df['_anchor_n'].astype(str)).values
def side_text_t(df):
    return (df['_context_n'].astype(str) + ' ' + df['_target_n'].astype(str)).values
def rowwise_cosine(Xa, Xt):
    Xa = l2_normalize(Xa, axis=1, copy=False)
    Xt = l2_normalize(Xt, axis=1, copy=False)
    sim = (Xa.multiply(Xt)).sum(axis=1)
    return np.asarray(sim).ravel()

dedup = pd.read_csv('artifacts/train_dedup_folds.csv')
y = dedup['score_mean'].values.astype(np.float32)
folds = dedup['fold'].values.astype(int)

prs_w = []
prs_c = []
for f in sorted(np.unique(folds)):
    tr_idx = np.where(folds != f)[0]
    va_idx = np.where(folds == f)[0]
    df_tr = dedup.iloc[tr_idx].reset_index(drop=True)
    df_va = dedup.iloc[va_idx].reset_index(drop=True)
    sA_tr = side_text_a(df_tr); sT_tr = side_text_t(df_tr)
    sA_va = side_text_a(df_va); sT_va = side_text_t(df_va)
    corpus = np.concatenate([sA_tr, sT_tr], axis=0)
    wvec = TfidfVectorizer(analyzer='word', ngram_range=(1,2), min_df=2, max_features=100_000)
    cvec = TfidfVectorizer(analyzer='char_wb', ngram_range=(3,5), min_df=2, max_features=200_000)
    wvec.fit(corpus); cvec.fit(corpus)
    cos_w = rowwise_cosine(wvec.transform(sA_va), wvec.transform(sT_va))
    cos_c = rowwise_cosine(cvec.transform(sA_va), cvec.transform(sT_va))
    pr_w = pearsonr(y[va_idx], cos_w)[0]
    pr_c = pearsonr(y[va_idx], cos_c)[0]
    prs_w.append(pr_w); prs_c.append(pr_c)
    print(f'Fold {f}: cos_word Pearson={pr_w:.4f}, cos_char Pearson={pr_c:.4f}, ranges: w[{cos_w.min():.3f},{cos_w.max():.3f}] c[{cos_c.min():.3f},{cos_c.max():.3f}]')

print('Mean cos_word Pearson:', float(np.mean(prs_w)))
print('Mean cos_char Pearson:', float(np.mean(prs_c)))

In [1]:
# Install PyTorch cu121 + transformers stack and sanity check GPU
import os, sys, subprocess, shutil, time
from pathlib import Path

def pip(*args):
    print('>', *args, flush=True)
    subprocess.run([sys.executable, '-m', 'pip', *args], check=True)

# Uninstall any preinstalled torch stack to avoid conflicts
for pkg in ('torch','torchvision','torchaudio'):
    subprocess.run([sys.executable, '-m', 'pip', 'uninstall', '-y', pkg], check=False)

# Clean stray site dirs that can shadow correct wheels (idempotent)
for d in (
    '/app/.pip-target/torch',
    '/app/.pip-target/torch-2.8.0.dist-info',
    '/app/.pip-target/torch-2.4.1.dist-info',
    '/app/.pip-target/torchvision',
    '/app/.pip-target/torchvision-0.23.0.dist-info',
    '/app/.pip-target/torchvision-0.19.1.dist-info',
    '/app/.pip-target/torchaudio',
    '/app/.pip-target/torchaudio-2.8.0.dist-info',
    '/app/.pip-target/torchaudio-2.4.1.dist-info',
    '/app/.pip-target/torchgen',
    '/app/.pip-target/functorch',
):
    if os.path.exists(d):
        print('Removing', d); shutil.rmtree(d, ignore_errors=True)

# Install exact cu121 torch stack
pip('install',
    '--index-url', 'https://download.pytorch.org/whl/cu121',
    '--extra-index-url', 'https://pypi.org/simple',
    'torch==2.4.1', 'torchvision==0.19.1', 'torchaudio==2.4.1')

# Freeze versions for later installs
Path('constraints.txt').write_text('torch==2.4.1\ntorchvision==0.19.1\ntorchaudio==2.4.1\n')

# Install transformer ecosystem (avoid upgrading torch)
pip('install', '-c', 'constraints.txt',
    'transformers==4.44.2', 'accelerate==0.34.2',
    'datasets==2.21.0', 'evaluate==0.4.2',
    'sentencepiece', 'scikit-learn', 'sentence-transformers==3.0.1',
    '--upgrade-strategy', 'only-if-needed')

# Sanity check GPU
import torch
print('torch:', torch.__version__, 'CUDA build:', getattr(torch.version, 'cuda', None))
print('CUDA available:', torch.cuda.is_available())
assert str(getattr(torch.version,'cuda','')).startswith('12.1'), f'Wrong CUDA build: {torch.version.cuda}'
assert torch.cuda.is_available(), 'CUDA not available'
print('GPU:', torch.cuda.get_device_name(0))

Found existing installation: torch 2.4.1+cu121


Uninstalling torch-2.4.1+cu121:
  Successfully uninstalled torch-2.4.1+cu121


Found existing installation: torchvision 0.19.1+cu121
Uninstalling torchvision-0.19.1+cu121:
  Successfully uninstalled torchvision-0.19.1+cu121


Found existing installation: torchaudio 2.4.1+cu121
Uninstalling torchaudio-2.4.1+cu121:
  Successfully uninstalled torchaudio-2.4.1+cu121
Removing /app/.pip-target/torch-2.4.1.dist-info
> install --index-url https://download.pytorch.org/whl/cu121 --extra-index-url https://pypi.org/simple torch==2.4.1 torchvision==0.19.1 torchaudio==2.4.1


Looking in indexes: https://download.pytorch.org/whl/cu121, https://pypi.org/simple


...

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.7/39.7 MB 170.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 144.3/144.3 KB 475.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 KB 484.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 509.2/509.2 KB 361.9 MB/s eta 0:00:00


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 229.9/229.9 KB 438.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 347.8/347.8 KB 525.4 MB/s eta 0:00:00


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 536.2/536.2 KB 492.9 MB/s eta 0:00:00


torch: 2.4.1+cu121 CUDA build: 12.1
CUDA available: False


AssertionError: CUDA not available